## <span style="color: #336699">Parte 03 - Aplicando o algoritmo em uma imagem Sentinel-2/A Nível 2</span>
<hr style="border:0.5px solid #0077b9;">


Agora, após a geração do nosso modelo nós iremos aplicar ele em uma imagem do Sentinel-2/MSI obtida através do STAC do BDC em Nível 2.

As imagems em nível 2 já são corrigidas dos efeitos atmosféricos. Porém, o método adotado pela ESA para o produto pronto não é totalmente adequado para aplicações em ambientes aquáticos, tendo alguns problemas. Nesse sentido, iremos avaliar qual é a diferença depois usando os métodos L2A (Sen2Cor) e ACOLITE. 
<br>

# 


In [ ]:
# Using RSTAC to get Sentinel-2 image and apply the algorithm developed

require(data.table)
require(dplyr)
require(rstac)
require(terra)
require(mapview)
require(httr)
require(rstac)
require(randomForest)
library(rasterVis)
require(RColorBrewer)
require(terrainr)

options(warn = -1)
options(repr.plot.width = 15, repr.plot.height = 15) 


In [ ]:

# We gonna use the BDC STAC to get Sentinel-2 images

stac_obj <- stac("https://data.inpe.br/bdc/stac/v1/")


In [ ]:

#
# query to the data catalog
#
catalog <- stac_obj %>% get_request()

print(catalog)


In [ ]:

#
# query the available product collections
#
collections <- stac_obj %>%
    collections() %>%
    get_request()

print(collections, n = 100)

In [ ]:
# Vamos usar sentinel-2 MSI Surface reflectance (inicialmente)


collection_info <- stac_obj %>%
    collections("S2_L2A-1") %>%
    get_request()

print(collection_info)


In [ ]:
dates = c("2025-03-05/2025-03-07")

BBOX = c(-46.632525, -23.770232, -46.6369, -23.758501) #xmin, ymin, xmax, ymax

it_obj <- stac_obj %>%
  stac_search(collections = "S2_L2A-1",
              bbox = BBOX,
              datetime = dates) %>%
  get_request() 

print(it_obj)



In [ ]:
## Filtrara para cloud-cover < 50 - se eu quiser...

item_filtered <- items_filter(it_obj, properties$`eo:cloud_cover` < 50)

print(item_filtered)



In [ ]:
## Carregando as bandas para o NDCI e para cor verdadeira


## Carregar os links dos assests (ou seja, os links para onde carregar as imagens)
b2.url  <- assets_url(item_filtered, asset_names = "B02", append_gdalvsi = TRUE) 
b3.url  <- assets_url(item_filtered, asset_names = "B03", append_gdalvsi = TRUE) 
b4.url  <- assets_url(item_filtered, asset_names = "B04", append_gdalvsi = TRUE) 
b5.url <- assets_url(item_filtered, asset_names = "B05", append_gdalvsi = TRUE) 
b12.url <- assets_url(item_filtered, asset_names = "B12", append_gdalvsi = TRUE) 
SCL.url <- assets_url(item_filtered, asset_names = "SCL", append_gdalvsi = TRUE) 

# Extensão do reservatório (poderia tmbém ser um shapefile, se quiser).

shp_billings = vect("Data/Billings_shape/billings.shp") %>% project(rast(b2.url))

# Cropar para area do reservatório apenas

b2 = rast(b2.url[1]) %>% crop(shp_billings)
b3 = rast(b3.url[1]) %>% crop(shp_billings)
b4 = rast(b4.url[1]) %>% crop(shp_billings)
b5 = rast(b5.url[1]) %>% project(b4)  %>% crop(shp_billings)

# Banda de nuvem
SCL = rast(SCL.url[1]) %>% project(b4)  %>% crop(shp_billings)



In [ ]:
plot(b2)

In [ ]:
mosaico = c(b2, b3, b4, b5)

names(mosaico) = c("B2", "B3", "B4", "B5")

# Remover as nuvens (Classes 7 8 e 9 são baixas, médias e altas probabilidades de nuvem)
mosaico[SCL == 7] = NA
mosaico[SCL == 8] = NA
mosaico[SCL == 9] = NA


plotRGB(mosaico, r = 3, g = 2, b = 1, stretch = 'lin')



In [ ]:
## Carregar o modelo de NDCI

modelo_NDCI = readRDS("Outputs/emp_chla.R")


## Criar a banda de NDCI no raster

mosaico$NDCI = ((mosaico$B5-mosaico$B4)/(mosaico$B5+mosaico$B4))+1

## aplicar a máscara de água do SCL

mosaico[SCL != 6] = NA

## Mosaico na água

plotRGB(mosaico, r = 3, g = 2, b = 1, stretch = 'lin')



In [ ]:
mosaico$Chla_NDCI = predict(mosaico, modelo_NDCI)

In [ ]:
colr <- colorRampPalette(rev(brewer.pal(11, 'RdBu')))

levelplot(raster::stack(mosaico$Chla_NDCI),col.regions = colr, 
          at = seq(0, 250, length = 100),
          maxpixels = 1e6, main = "Chlorophyll-a Concentration (ug/L)") 

In [ ]:
writeRaster(mosaico$Chla_NDCI, "Outputs/chla_sen2cor.tif", overwrite = T)